# Joins and Subqueries - SQL Notebook

## Overview
This Jupyter Notebook contains an exploration of SQL joins and subqueries, providing practical examples and explanations. It is designed to help learners understand how to retrieve and manipulate data across multiple tables efficiently.

## Contents
- **SQL Joins**: Inner, Left, Right, and Full Joins
- **Subqueries**: Nested queries for filtering, aggregation, and transformation
- **Use Cases**: Practical scenarios demonstrating when to use joins vs. subqueries
- **Example Queries**: Hands-on SQL queries with explanations
- **Performance Considerations**: Tips on optimising queries for better performance

## How to Use This Notebook
- Run each cell sequentially to execute SQL queries
- Modify the example queries to experiment with different joins and subqueries
- Review the explanations to understand the logic behind each query
- Use the provided dataset or connect to your own database for practice



In [2]:
%load_ext sql

%sql postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics

Find all the athletes who won a medal by joining the athletes and medals tables

In [19]:
%%sql

SELECT athletes.name, athletes.country, athletes.sport, * FROM athletes INNER JOIN medals ON athletes.name = medals.name AND athletes.sport = medals.sport
LIMIT 3



 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


name,country,sport,index,name_1,country_1,countrycode,sport_1,sport_code,date_of_birth,gender,index_1,country_2,countrycode_1,name_2,sport_2,sport_short,event,medal
ABARZA Alberto,Chile,Swimming,1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male,1156,Chile,CHI,ABARZA Alberto,Swimming,SWM,Men's 50m Backstroke - S2,Silver Medal
ABARZA Alberto,Chile,Swimming,1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male,1126,Chile,CHI,ABARZA Alberto,Swimming,SWM,Men's 200m Freestyle - S2,Silver Medal
ABARZA Alberto,Chile,Swimming,1,ABARZA Alberto,Chile,CHI,Swimming,SWM,1984-12-11,Male,393,Chile,CHI,ABARZA Alberto,Swimming,SWM,Men's 100m Backstroke - S2,Gold Medal


In [4]:
%%sql

SELECT COUNT(*) FROM medals

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1 rows affected.


count
2142


Who was the oldest medal winner?

In [5]:
%%sql

-- Who was the oldest medal winner?


SELECT * FROM athletes 
RIGHT JOIN medals 
ON athletes.name = medals.name 
AND athletes.sport = medals.sport
ORDER BY date_of_birth
LIMIT 1

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1 rows affected.


index,name,country,countrycode,sport,sport_code,date_of_birth,gender,index_1,country_1,countrycode_1,name_1,sport_1,sport_short,event,medal
1266,FREI Heinz,Switzerland,SUI,Cycling Road,CRD,1958-01-28,Male,923,Switzerland,SUI,FREI Heinz,Cycling Road,CRD,Men's H3 Road Race,Silver Medal


Who was the youngest medal winner?

In [6]:
%%sql

SELECT * FROM athletes 
RIGHT JOIN medals 
ON athletes.name = medals.name 
AND athletes.sport = medals.sport
ORDER BY date_of_birth DESC
LIMIT 1

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1 rows affected.


index,name,country,countrycode,sport,sport_code,date_of_birth,gender,index_1,country_1,countrycode_1,name_1,sport_1,sport_short,event,medal
4346,YAMADA Miyuki,Japan,JPN,Swimming,SWM,2006-09-15,Female,1173,Japan,JPN,YAMADA Miyuki,Swimming,SWM,Women's 100m Backstroke - S2,Silver Medal


Use a SELF JOIN to find athletes that share a birthday

In [ ]:
%%sql

-- Atempt 1: find two athletes that share a birthday

SELECT DISTINCT alias_a.name athlete_1, alias_b.name athlete_2
FROM athletes alias_a 
INNER JOIN athletes AS alias_b 
ON alias_a.date_of_birth = alias_b.date_of_birth
AND alias_a.name < alias_b.name
ORDER BY athlete_1
LIMIT 4


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
4 rows affected.


athlete_1,athlete_2
ABARZA Alberto,CARDOSO da SILVA Luis Carlos
ABDELALI Samiha,PEREIRA Pamela
ABDELHADY Hany,KRUTOVA Elena
ABDELSHAFI Malak,JENKINS Mikaela


In [ ]:
%%sql

--Attempt 2: find the number of athletes that have birthdays on each day

SELECT date_of_birth, COUNT(*) AS num_athletes
FROM athletes
GROUP BY date_of_birth
HAVING COUNT(*) > 1
ORDER BY num_athletes DESC
LIMIT 3



 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


date_of_birth,num_athletes
1999-02-05,5
1995-10-11,5
1989-10-05,4


In [ ]:
%%sql 

-- Attempt 3: build a list of all athletes that share a birthday

SELECT date_of_birth AS shared_birthday, 
       STRING_AGG(name, ', ') AS athletes
FROM athletes
GROUP BY date_of_birth
HAVING COUNT(*) > 1
LIMIT 3


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


shared_birthday,athletes
1964-07-05,"SIM Jae Yong, CHEBANIKA Raisa"
1966-10-21,"GARRIDO MARQUEZ Victor Hugo, GARRIDO MARQUEZ Victor Hugo"
1967-10-12,"SCHNARNDORF Susana, PILLE-STEPPAT Sylvia"


Write a query that generates a table like this...

COLUMNS: 
country	athlete_count	medal_count	medals_per_athlete

ROWS:
People's Republic of China	256	276	1.08
Netherlands	74	77	1.04
Italy	114	91	0.80
Ukraine	139	108	0.78
Great Britain	221	171	0.77

In [22]:
%%sql

SELECT a.country, a.athlete_count, m.medal_count, ROUND(1.0 * m.medal_count / a.athlete_count, 2) AS medals_per_athlete
FROM
(
SELECT country, COUNT(*) AS athlete_count
FROM athletes
GROUP BY country
) as a
JOIN
(
SELECT country, COUNT(*) AS medal_count
FROM medals
GROUP BY country
) as m
ON a.country = m.country
ORDER BY medals_per_athlete DESC
LIMIT 3


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


country,athlete_count,medal_count,medals_per_athlete
People's Republic of China,256,276,1.08
Netherlands,74,77,1.04
Italy,114,91,0.80


In [12]:
%%sql 

SELECT name, sport,
DATE_PART('year', CURRENT_DATE) - DATE_PART('year', date_of_birth) as age
FROM athletes
LIMIT 3

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


name,sport,age
AAJIM Munkhbat,Judo,36.0
ABARZA Alberto,Swimming,41.0
ABASLI Namig,Judo,28.0


Temu wanted to write a query that generates a table showing the number of athletes and gold medals for each country. Their code is below - it doesn't work as intended.

In [13]:
%%sql

-- medals should be gold_medals in line 1
-- country in the medals_sub table should be matched with country in athletes_sub table, not countrycode


-- Corrected implementation:

SELECT medals_sub.country, medals_sub.gold_medals, athletes_sub.athletes 
FROM (
    SELECT COUNT(*) as gold_medals, country
    FROM medals
    WHERE medal='Gold Medal'
    GROUP BY country
) medals_sub
INNER JOIN (
    SELECT COUNT(*) as athletes, country 
    FROM athletes 
    GROUP BY country
) athletes_sub
ON medals_sub.country = athletes_sub.country
LIMIT 3

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


country,gold_medals,athletes
Indonesia,4,23
Venezuela,3,27
Czech Republic,2,29


Paula wanted to write a query that generates a table showing the average age of competitors winning each medal - gold, silver or bronze. Her query is below - does it work as intended? HINT: The answer is no :)

Outcome: columns - gold, silver bronze, rows - average age of winner



In [14]:
%%sql

SELECT subquery_result.medal, (SUM(subquery_result.age)/COUNT(subquery_result.age)) as average_age
FROM (
  SELECT DISTINCT ON (athletes.name, medals.medal)
    medals.medal,
    DATE_PART('year', CURRENT_DATE) - DATE_PART('year', athletes.date_of_birth) as age
    FROM athletes INNER JOIN medals
    ON athletes.name = medals.name
) subquery_result
GROUP BY medal;

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
3 rows affected.


medal,average_age
Gold Medal,33.955516014234874
Bronze Medal,35.2034632034632
Silver Medal,34.689542483660134


In [15]:
%%sql


SELECT (SUM(age) / COUNT(age)) average_age
FROM (
SELECT DISTINCT athletes.name, DATE_PART('year', CURRENT_DATE) - DATE_PART ('year', athletes.date_of_birth) AS age, medals.medal FROM athletes
INNER JOIN medals
ON athletes.name = medals.name
WHERE medal = 'Gold Medal'
) gold_medals
GROUP BY medal;
 

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1 rows affected.


average_age
33.96092362344583
